In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor

%matplotlib inline

In [2]:
!ls Data

meta_Beauty.json.gz      reviews_Beauty_5.json.gz


In [3]:
df = pd.read_json("Data/reviews_Beauty_5.json.gz",lines=True)

In [4]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [5]:
df.isna().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [6]:
#1386 reviewerNames left blank; we will not need reviewerName since utilizing reviewerID but 
#curious why

In [7]:
df[df['reviewerName'].isnull()]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
1790,AK1H26O8DLMNN,B0000535UM,NaN,"[0, 0]",The first thickening shampoo that works on my ...,5,Actually works,1405123200,"07 12, 2014"
2242,APTLHR9PHGPXN,B00005NAOD,NaN,"[0, 0]","Kind of drying, not moisturizing. Kind of disa...",2,Inexpensive and feels that wY,1405209600,"07 13, 2014"
2304,AQWX644AFUFFK,B00005NFBD,NaN,"[0, 0]","This is just ok. For one, I found this in a st...",3,"Ok, nothing amazing",1405468800,"07 16, 2014"
3651,A43K5ZRQ87TO6,B00008PC1O,NaN,"[0, 0]",Works well and easy to use!,5,Five Stars,1405296000,"07 14, 2014"
...,...,...,...,...,...,...,...,...,...
197192,A1Z3AV93ONK5VF,B00KAL5JAU,NaN,"[0, 0]",We already had the Dead Sea Shampoo by Adovia ...,5,"Non greasy, silky shiny hair",1401235200,"05 28, 2014"
197193,A184I8GT3BHZQV,B00KAL5JAU,NaN,"[0, 1]",&#60;a href=&#34;http://www.tomoson.com/?code=...,5,Dead Sea Salt hair conditioner,1403568000,"06 24, 2014"
197194,A8C9EJORQD23,B00KAL5JAU,NaN,"[0, 1]",I use this with the Adovia shampoo I mention a...,5,Adovia does natural right!,1402272000,"06 9, 2014"
198446,A2PIGZCDGM4NJ7,B00L5JHZJO,NaN,"[10, 11]","This is a horrible product, most of the review...",1,Bad Product,1404864000,"07 9, 2014"


In [8]:
#all necessary information is included even when reviewerName is NaN

In [9]:
df.reviewerID.value_counts()

A2V5R832QCSOMX    204
ALNFHVS3SC4FV     192
AKMEY1BSHSDG7     182
A3KEZLJ59C1JVH    154
ALQGOMOY1F5X9     150
                 ... 
A87PSJYV6AKPQ       5
A2WXZFI0R49CNE      5
A3IG80M5WPK85G      5
A3IG2S1UPOE0ZS      5
A557GAAFX4L21       5
Name: reviewerID, Length: 22363, dtype: int64

In [10]:
#YAY! all reviewerIDs have value of at least 5, total of 22,363 reviewers

In [11]:
df.asin.value_counts()

B004OHQR1Q    431
B0043OYFKU    403
B0069FDR96    391
B000ZMBSPE    389
B00150LT40    329
             ... 
B001CL2L6M      5
B0042I19QU      5
B004UNH29U      5
B001QOP2KM      5
B000FPPW9O      5
Name: asin, Length: 12101, dtype: int64

In [12]:
#YAY! all products have at least 5 reviews, total of 12,101 different products

In [13]:
#looking at ratings distribution

In [14]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = df['overall'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} Reviews'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [15]:
# Number of ratings per product
data = df.groupby('asin')['overall'].count().clip(upper=50)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Product (Clipped at 100)',
                   xaxis = dict(title = 'Number of Ratings Per Product'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [16]:
#We see a majority of our projects have 10 or less ratings

In [17]:
df.groupby('asin')['overall'].count().reset_index().sort_values('overall', ascending=False)[:10]

,asin,overall
7531,B004OHQR1Q,431
6726,B0043OYFKU,403
8823,B0069FDR96,391
2647,B000ZMBSPE,389
2986,B00150LT40,329
6434,B003V265QW,328
8968,B006L1DNWY,321
10162,B008U1Q4DI,310
9368,B007BLN17K,305
267,B000142FVW,302


In [18]:
#10 products with highest # of ratings

In [19]:
#Ratings Distribution by user

In [20]:
# Number of ratings per user
data = df.groupby('reviewerID')['overall'].count().clip(upper=50)

trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 50)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [21]:
#Most users rated under 10 products

In [22]:
#can save data as a CSV file for easier use, but it does not look like we need to clean data
#can drop reviewerNames

In [23]:
lower_rating = df.overall.min()

In [24]:
upper_rating = df.overall.max()

In [25]:
print('Review range: {0} to {1}'.format(lower_rating, upper_rating))

Review range: 1 to 5


In [26]:
#Confirming our review range is 1 to 5, which is the default for surprise

In [27]:
#Creating dataframe with appropriate columns to run through surprise

In [28]:
surprise_df = df[['reviewerID', 'asin', 'overall']]

In [29]:
surprise_df

,reviewerID,asin,overall
0,A1YJEY40YUW4SE,7806397051,1
1,A60XNB876KYML,7806397051,3
2,A3G6XNM240RMWA,7806397051,4
3,A1PQFP6SAJ6D80,7806397051,2
4,A38FVHZTNQ271F,7806397051,3
...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,5
198498,A1UQBFCERIP7VJ,B00LLPT4HI,5
198499,A35Q0RBM3YNQNF,B00LLPT4HI,5
198500,A3LGT6UZL99IW1,B00LLPT4HI,5


In [30]:
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(surprise_df, reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=23)

In [31]:
surprise_data

In [32]:
# How many users and items are in the trainset
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  22359 

Number of items:  12101 



In [33]:
#10,000 more users than items

In [34]:
print('Type trainset :',type(trainset),'\n')
print('Type testset :',type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


In [35]:
print(len(testset))
print(testset[0])

39701
('A32POM4ALTYIZV', 'B005Z4QT7E', 2.0)


In [36]:
print(len(df))

198502


In [37]:
198502 * .2

39700.4

In [38]:
#sanity check that test set is 20% of total data

In [39]:
#starting with item-item similarity

In [40]:
sim_cos = {'name':'cosine', 'user_based':False}

In [41]:
from surprise.prediction_algorithms import knns

In [42]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [43]:
# looking at the similarity metrics of each of the items to one another by using the sim attribute of our fitted model

In [44]:
basic.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [45]:
predictions = basic.test(testset)

In [46]:
print(accuracy.rmse(predictions))

RMSE: 1.2129
1.2128933192100995


In [47]:
#RMSE of about 1.2, meaning that it was off by roughly 1.2 points for each guess it made for ratings

In [48]:
sim_cos_user = {'name':'cosine', 'user_based':True}

In [49]:
basic_user = knns.KNNBasic(sim_options=sim_cos_user)
basic_user.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [50]:
basic_user.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [51]:
predictions_user = basic_user.test(testset)

In [52]:
print(accuracy.rmse(predictions_user))

RMSE: 1.2401
1.2400588428109054


In [53]:
#KNN model with Pearson Correlation 

In [54]:
sim_pearson = {'name':'pearson', 'user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1854
1.1854039491870065


In [55]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
basic_pearson_user = knns.KNNBasic(sim_options=sim_pearson_user)
basic_pearson_user.fit(trainset)
predictions_user = basic_pearson_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1795
1.1795381227887218


In [56]:
#KNN model with Means

In [57]:
#same thing as the basic KNN model, except it takes into account the mean rating of each user or item depending on 
#whether you are performing user-user or item-item similarities, respectively.

In [58]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1791
1.1790736593254243


In [59]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
knn_means_user = knns.KNNWithMeans(sim_options=sim_pearson_user)
knn_means_user.fit(trainset)
predictions_user = knn_means_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1622
1.162184545973618


In [60]:
#This is our best model so far

In [61]:
#Running an SVD model with defaults

In [62]:
svd = SVD(random_state=42)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.0889
1.0889451149217502


In [63]:
#Lowest RMSE so far!

In [64]:
svd.predict('A1YJEY40YUW4SE', 'B00LLPT4HI')

Prediction(uid='A1YJEY40YUW4SE', iid='B00LLPT4HI', r_ui=None, est=4.41104541567184, details={'was_impossible': False})

In [65]:
svd.predict('A2BLFCOPSMBOZ9', '7806397051')

Prediction(uid='A2BLFCOPSMBOZ9', iid='7806397051', r_ui=None, est=3.774499861592997, details={'was_impossible': False})

In [66]:
#Checking to see estimated rating for 2 user/product combinations

In [67]:
trainset

In [68]:
cv_svd_baseline = cross_validate(svd, surprise_data)

In [69]:
cv_svd_baseline

{'test_rmse': array([1.09023229, 1.08569982, 1.08943776, 1.09413354, 1.09110017]),
 'test_mae': array([0.83843032, 0.83052222, 0.8339665 , 0.83890543, 0.83740998]),
 'fit_time': (9.302681922912598,
  9.532520294189453,
  9.54220199584961,
  9.5253267288208,
  9.684647798538208),
 'test_time': (0.20006084442138672,
  0.19495201110839844,
  0.4689140319824219,
  0.197767972946167,
  0.475740909576416)}

In [70]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [71]:
#Baseline Models; true baseline should be NormalPredictor(); user_based=True

In [72]:
baseline = NormalPredictor()
baseline.fit(trainset)

In [73]:
predictions = baseline.test(testset)

In [74]:
baseline = accuracy.rmse(predictions)

RMSE: 1.5000


In [75]:
baseline2 = BaselineOnly()
baseline2.fit(trainset)

Estimating biases using als...


In [76]:
predictions2 = baseline2.test(testset)

In [77]:
baseline2 = accuracy.rmse(predictions2)

RMSE: 1.0890


In [78]:
##Baseline models using ALS, SGD

In [79]:
als_options = {'method': 'als',
               }
als_baseline = BaselineOnly(bsl_options=als_options)

In [80]:
als_baseline.fit(trainset)

Estimating biases using als...


In [81]:
predictions = als_baseline.test(testset)

In [82]:
als_baseline = accuracy.rmse(predictions)

RMSE: 1.0890


In [83]:
sgd_options = {'method': 'sgd',
               }
sgd_baseline = BaselineOnly(bsl_options=sgd_options)

In [84]:
sgd_baseline.fit(trainset)

Estimating biases using sgd...


In [85]:
predictions = sgd_baseline.test(testset)

In [86]:
sgd_baseline = accuracy.rmse(predictions)

RMSE: 1.0818


In [87]:
#More KNN models

In [88]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
knn_z = knns.KNNWithZScore(sim_options=sim_pearson_user)
knn_z.fit(trainset)
predictions = knn_z.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1615
1.161535407286978


In [89]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson_user)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1068
1.1068094494514653


In [90]:
#sgd baseline performing the best

In [91]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
knn_means2 = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means2.fit(trainset)
predictions = knn_means2.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1791
1.1790736593254243


In [92]:
bsl_options = {'method': 'als',
               'n_epochs': 20,
               }
sim_options = {'name': 'pearson_baseline'}
basic_als = knns.KNNBasic(bsl_options=bsl_options, sim_options=sim_options)
basic_als.fit(trainset)
predictions=basic_als.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.1777
1.177664471541867


In [93]:
bsl_options = {'method': 'sgd'
               }
sim_options = {'name': 'pearson_baseline'}
basic_sgd = knns.KNNBasic(bsl_options=bsl_options, sim_options=sim_options)
basic_sgd.fit(trainset)
predictions=basic_sgd.test(testset)
print(accuracy.rmse(predictions))

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.1790
1.1790476357494852


In [94]:
sgd_options = {'method': 'sgd',
               }
sgd_baseline = BaselineOnly(bsl_options=sgd_options)
sgd_baseline.fit(trainset)
predictions = sgd_baseline.test(testset)
sgd_baseline = accuracy.rmse(predictions)

Estimating biases using sgd...
RMSE: 1.0818


In [95]:
#BaselineOnly still best model.... KNNBaseline next best

In [96]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

In [97]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, surprise_data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,1.094251,24.364677,1.485184
BaselineOnly,1.095756,0.336231,0.563342
SVD,1.096776,7.876873,0.434228
KNNBaseline,1.168872,19.785385,2.896681
CoClustering,1.206526,3.869398,0.212355
KNNWithMeans,1.213236,19.305426,2.626993
KNNWithZScore,1.221557,19.720624,2.710014
KNNBasic,1.231031,19.345290,2.790209
SlopeOne,1.245007,3.657140,0.896170


In [98]:
# New dictionary for SVD++
svdpp_param_grid = {'n_factors':[10, 20],
                    'n_epochs':[20, 30],
                    'reg_all':[0.02, 0.05], 
                    "lr_all": [0.007, 0.005]}
#svdpp_gs_model = GridSearchCV(SVDpp, param_grid=svdpp_param_grid, cv=3, joblib_verbose=10, n_jobs=-1, return_train_measures=True)

# Fit and return the best_params based on cross validation this will take a VERY long time to run
#svdpp_gs_model.fit(surprise_data)
#svdpp_gs_model.best_params['rmse']

In [99]:
#{'n_factors': 10, 'n_epochs': 20, 'reg_all': 0.05, 'lr_all': 0.005}

In [100]:
# Instantiate - fit on trainset - score the model on testset
SVDpp_model = SVDpp(n_factors=10, n_epochs=20, random_state=42, reg_all=0.05, lr_all=.005)
SVDpp_model.fit(trainset)
predictions = SVDpp_model.test(testset)
SVDpp_gs = accuracy.rmse(predictions)

RMSE: 1.0823


In [101]:
#RMSE: 1.0823

In [102]:
# New dictionary for SVD++
svdpp_param_grid = {'n_factors':[15, 20, 25],
                    'n_epochs':[10, 20 ],
                    'reg_all':[0.02, 0.05, .07], 
                    "lr_all": [0.007, 0.005, .002]}
#svdpp_gs_model = GridSearchCV(SVDpp, param_grid=svdpp_param_grid, cv=3, joblib_verbose=10, n_jobs=-1, return_train_measures=True)

# Fit and return the best_params based on cross validation this will take a VERY long time to run
#svdpp_gs_model.fit(surprise_data)
#svdpp_gs_model.best_params['rmse']

In [103]:
#{'n_factors': 15, 'n_epochs': 20, 'reg_all': 0.07, 'lr_all': 0.005}#

In [104]:
# Instantiate - fit on trainset - score the model on testset
SVDpp_model = SVDpp(n_factors=15, n_epochs=20, random_state=42, reg_all=0.07, lr_all=.005)
SVDpp_model.fit(trainset)
predictions = SVDpp_model.test(testset)
SVDpp_gs = accuracy.rmse(predictions)

RMSE: 1.0824


In [105]:
#RMSE: 1.0824
#still not as good as sgd baseline

In [106]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [107]:
avg_rating_user = df.groupby("reviewerID")["overall","reviewerID"].mean().sort_values("overall")
avg_rating_user

<ipython-input-107-8752c1212ea2>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,overall
reviewerID,
A1W522Z24EPBJB,1.0
A2DPSPXFJ507C0,1.0
A1GQLVT0SWAWU,1.0
A1KLA02LZXAT46,1.0
A2MHHSACEJANSX,1.0
...,...
A15QGN6UXJVW9G,5.0
ANOJX4RAUJ9HL,5.0
A2RJT3IE2T6KXJ,5.0


In [108]:
low_rating_user = avg_rating_user[avg_rating_user["overall"]==1.0]
low_rating_user

,overall
reviewerID,
A1W522Z24EPBJB,1.0
A2DPSPXFJ507C0,1.0
A1GQLVT0SWAWU,1.0
A1KLA02LZXAT46,1.0
A2MHHSACEJANSX,1.0
A2RJTIE73NPN3C,1.0
ASWIC85F71H4J,1.0
A2TBE0N8JN6H4K,1.0
A1GPPMHYM6SMEW,1.0


In [109]:
high_rating_user = avg_rating_user[avg_rating_user["overall"]==5.0]
high_rating_user

,overall
reviewerID,
A2FINIRQNXOTI,5.0
ATWS89FH6Y6S4,5.0
A16Q479PYT0G6N,5.0
A3OKW5VRXZG3OQ,5.0
A3O9Q3154FPZLL,5.0
...,...
A15QGN6UXJVW9G,5.0
ANOJX4RAUJ9HL,5.0
A2RJT3IE2T6KXJ,5.0


In [110]:
high_rating_user_ids = list(high_rating_user.index)
high_rating_user_ids

['A2FINIRQNXOTI',
 'ATWS89FH6Y6S4',
 'A16Q479PYT0G6N',
 'A3OKW5VRXZG3OQ',
 'A3O9Q3154FPZLL',
 'A2FJRU6RKAEAYX',
 'A1I9YRWNWRO3BY',
 'A2FQ1Q3V31QLQB',
 'A8WJERXUE7P43',
 'A2FAVDANCZYKNX',
 'A3OK7MGXBAK4JM',
 'A1YUGBWG0G9REI',
 'A1BOW2E468Y7Q1',
 'A8TPE62B0UCCE',
 'A2FKG99KYKTZQO',
 'A3OQ4I8U6TYCVE',
 'A3OOIWLW0Y01C2',
 'A2FLXE8YW1LMBN',
 'A1BPCAYEK3T1V7',
 'A2FGQ0JGXYYW6D',
 'A2FM0GJWDCQNFD',
 'A8QXLDP4JBE3K',
 'A3OPVTWOL57FCW',
 'A2FM5AJ7QH33S0',
 'A2FOR1IJSX76LQ',
 'A3OP6HJMA7SKDB',
 'A2FI3GF4Z7QNO',
 'A3OHN0JHXJ9BLA',
 'A2FEJKLHXJY7JT',
 'A1BO0E7TFN58TS',
 'A1YX68282RPR0S',
 'A2FC89NUP5CL8B',
 'A16Q29GKE0WNW4',
 'A3OD3HNHVSYJCR',
 'ATVAMZS8V7AZ6',
 'ATZFZONPLXQJN',
 'A3O27SJDAU2UAG',
 'ATSVUBE2115N1',
 'A3NPO4ECDIMF92',
 'A2GFS17LT7K6QV',
 'ATFF039E40YCS',
 'A1YQZCX2K3ISX5',
 'A94FW86TF1HL7',
 'A1YRHLWLQYRXKH',
 'A3NSLQZTVCO7J6',
 'A3NSLW1M3469ST',
 'A2GDP615CA6E9J',
 'ATH3U4X61L8D5',
 'A2GD4LJFKOZ9YZ',
 'A93DR0MWWC2B7',
 'ATHI1A9N10E6A',
 'A930TA3RDIJRS',
 'ATHP3KFKOKX6Z',
 'A16KNLK

In [111]:
df_cleaned = df[~df.reviewerID.isin(high_rating_user_ids)]
df_cleaned

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [112]:
#Testing a few high rating user IDs to see if we succesfully dropped from our cleaned df
df_cleaned[df_cleaned["reviewerID"]== "A3OOIWLW0Y01C2"]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime


In [113]:
#It worked!
df_cleaned[df_cleaned["reviewerID"]== "A2FM0GJWDCQNFD"]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime


In [115]:
df_cleaned

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [157]:
df_cleaned['asin'].value_counts(ascending=True)[:1170]

B00GPQUFUI    1
B00IX4MF76    1
B00380DF3S    1
B004813W5M    1
B00IC7L3JK    1
             ..
B000X81832    4
B0089YHZ4I    5
B00C0HF1J6    5
B0015Z7IH2    5
B003UTRNXK    5
Name: asin, Length: 1170, dtype: int64

In [163]:
#We see when we cleaned our reviewers, we now have many products with under 5 reviews
#approx 1165 products with 1-4 reviews

In [164]:
#how to deploy model

In [ ]:
#get rid of items that the user has already rated

In [165]:
#.predict will take in UserID; can map name to UniqueID (ReviewerName)

In [167]:
#use .predict to create new column with estimated rating for every item in datasert

In [168]:
#sort df/list by estimated rating ,return n # of recommendations

In [169]:
#do you want to return just product name, product #, details, etc.